![Podmanlogo](Pictures/podman-logo.png)

# Running containers at system start

There are many situations in which you may want to start a container automatically when the system starts. A good example of this would be an edge environment, think of a small ruggered computer in a factory that runs containerized software using Podman. If there is a power failure the system will of course shut down, once the power is back the system will boot and most likely you would like your containers to start automatically as well. We can easily achieve this by taking advantage of systemd and Podman.

In most linux distributions systemd is the parent of all processes on the system, it is executed by the kernel and is responsible for starting all other processes. If we want to automatically start a process (like a container), at system start it's a best practice to take advantage of systemd. Podman is capable of helping with this process, lets see how.

We will start deploying a container with Podman the same way we've been doing it during the rest of the workshop.

In [ ]:
podman run --rm -d --name my-autostart-container fedora sleep 99999

Check the container is properly deployed

In [ ]:
podman ps -a

Our container is running and it's seen by the operating system as a process. Now we would need to generate a systemd unit file that systemd can use to execute this process when the system starts. Podman provides a very easy way of generating this file from a running container:

In [ ]:
podman generate systemd --new --files --name my-autostart-container

As you can see the output explains that this is a deprecated command, we will come back to that in a minute. First lets take a look at the output file that was generated.

In [ ]:
cat container-my-autostart-container.service

As you can see Podman has generated the file for us, the only additional thing we would need to do now is to place this file in ls "$HOME/.config/systemd/user" and reload systemd running "systemctl --user daemon-reload". Once you've done that your system will automatically start your container on boot.

This is cool but, as you've seen, the generated file is long and complex to read specially for non experienced users. Also, most of the times we work with containers we want to work in a declarative manner. This means we would like to have a file in which we define how our application should run and an orchestrator will make it work, this is the default way of working with kubernetes and has become a de-facto standard for containerized environments.

For these reasons Podman introduced Quadlets in version 4.4. This technology brings the capability of writting a simple and easy to read file that can be used how you want you container to run (following the declarative model). Lets see how it works.

The only thing you need to do to take advantage of Quadlets is creating a symple file with the definition of your container and with the name ending in ".container":

In [ ]:
cat << EOF > my-quadlet.container
[Unit]
Description=The sleep container
After=local-fs.target

[Container]
Image=docker.io/library/fedora:latest
Exec=sleep 1000

[Install]
# Start by default on boot
WantedBy=multi-user.target default.target
EOF
cat my-quadlet.container


As you can see this is a very easy to read file that allows you to take advantage of the declarative way of working. Once you have created this file you just need to move it to $HOME/.config/containers/systemd/ if you're a rootless user or to /usr/share/containers/systemd/ if you have root access to the system. Once you've done that reload systemd by running "systemctl --user daemon-reload". Let's do it.

In [ ]:
mkdir ~/.config/containers/systemd/ -p
mv ./my-quadlet.container ~/.config/containers/systemd/
systemctl --user daemon-reload

Now we can use systemctl to start your container:

In [ ]:
systemctl --user start my-quadlet.service

And also check it's status:

In [ ]:
systemctl --user status my-quadlet.service --no-pager

You can also stop the service and that will kill the Podman container. Before testing it check that your container is running with Podman:

In [ ]:
podman ps

Then stop the systemd unit:

In [ ]:
systemctl --user stop my-quadlet.service

Now check wit Podman again:

In [ ]:
podman ps -a

As you can see, Quadlets transform your containers in a systemd unit that can be managed in a super easy way.

If you rebooted this system you'd see the container is automatically executed on boot. Be aware this workshop runs using a shared environment between multiple students, so booting off the system is not allowed.

# Automatic boot of Patient Portal application

We have seen how to use Quadlets to boot a single and very simple container. Now let's see what we would need to do to run the whole Patient Portal application.

In case you forgot, this is how the architecture of the application looks like:

![PatientPortalApplication](Pictures/patient-portal-application-storage.png)

We will not use a pod as we did in the last section because our application is not designed to be used that way. However, you can also manage pods with Quadlets, if you're interested in it please take a look at the documentation.

Before we spin up any container of our application we need to have all the networks and volumes. This can be defined with Quadlets whose file name ends in ".network" or ".volume". In our case those files will be mostly empty as we don't need to configure anything special for any of them.

Begin by creating the Quadlet file for payment network:

In [ ]:
cat << EOF > payment.network
[Network]
EOF
cat payment.network

The one for the database network:

In [ ]:
cat << EOF > database.network
[Network]
EOF
cat database.network

And now the volume:

In [ ]:
cat << EOF > patient-portal-data.volume
[Volume]
EOF
cat patient-portal-data.volume

These definitions may look empty but that is enough for the Quadlets to know they need to create the networks and the volume with the standard configuration. Of course you could add detailed information for advanced configuration.

Now it's time to start with the container Quadlet files, lets begin with the database container:

In [ ]:
cat << EOF > database.container
[Install]
WantedBy=default.target

[Container]
Image=quay.io/skupper/patient-portal-database
ContainerName=database
Volume=patient-portal-data.volume:/var/lib/postgresql/data
Network=database.network
EOF
cat database.container

Then the file for the payment processor:

In [ ]:
cat << EOF > payment-processor.container
[Install]
WantedBy=default.target

[Unit]
Requires=database.service
After=database.service

[Container]
Image=quay.io/skupper/patient-portal-payment-processor
ContainerName=payment-processor
Network=payment.network
EOF
cat payment-processor.container

Pay attention to the "[Unit]" section in previous file. With the fields "Requires" and "After" we are specifying that this container needs to be deployed after the database container. By doing this we guarantee there is enough time for the database to initialize before the frontend container is deployed, otherwise all three containers would start at the same time and the frontend would fail because it wouldn't be able to reach the database.

Last one, the frontend container:

In [ ]:
cat << EOF > frontend.container
[Install]
WantedBy=default.target

[Unit]
Requires=payment-processor.service
After=payment-processor.service

[Container]
Image=quay.io/skupper/patient-portal-frontend
ContainerName=frontend
Network=payment.network
Network=database.network
PublishPort=8080:8080
Environment=DATABASE_SERVICE_HOST="database"
Environment=DATABASE_SERVICE_PORT="5432"
Environment=PAYMENT_PROCESSOR_SERVICE_HOST="payment-processor"
Environment=PAYMENT_PROCESSOR_SERVICE_PORT="8080"
EOF
cat frontend.container

Here you can see how to pass environment variables and values to the container. This helps using the same container image for different purposes, providing great flexibility.
Also pay attention to the fact that we added both networks to the container definition so it can reach the database and the payment processor.
Last important line to notice is the one in which we specified the port in the container to be exposed by using the "PublishPort" option.

With all these files, next thing we have to do is to copy them to the correct directory and reload systemd daemon.

In [ ]:
mv ./* ~/.config/containers/systemd/
systemctl --user daemon-reload

We can just the systemctl tool to start our containers. Remember we added dependencies so the frontend only starts after the other two containers are deployed? Lets check how that works by trying to start the frontend Quadlet.

In [ ]:
systemctl --user start frontend.service

It fails as expected, we can see the details looking at linux journal:

In [ ]:
journalctl -xe | tail

Now let's do it in the proper order, start the database container and check it's status:

In [ ]:
systemctl --user start database.service
systemctl --user status database.service --no-pager

Then the payment processor container:

In [ ]:
systemctl --user start payment-processor.service
systemctl --user status payment-processor.service --no-pager

And lastly the frontend container:

In [ ]:
systemctl --user start frontend.service
systemctl --user status frontend.service  --no-pager

Check all the containers are running:

In [ ]:
podman ps

Check logs on the frontend container to be sure everything is working fine:

In [ ]:
podman logs frontend

And check the application answers an http request:

In [ ]:
curl -s localhost:8080

As you can see everything is working perfectly fine and in a very easy manner!

Later in this workshop we will also see how deep the integration between Podman and kubernetes is, we'll show how Quadlets are able to manage kubernetes yaml files 

# Cleanup

In [ ]:
systemctl --user stop frontend.service
systemctl --user stop payment-processor.service
systemctl --user stop database.service
rm ~/.config/containers/systemd/* -rf
systemctl --user daemon-reload
podman rm --all -f
podman network prune -f
podman volume prune -f
podman pod prune -f
podman image prune -f